## Receta de personalización del usuario

Este notebook muestra cómo utilizar la nueva receta de personalización de usuarios de Amazon Personalize (aws-user-personalization). Esta receta equilibra las recomendaciones entre elementos nuevos y antiguos, lo que le permite ajustar el equilibrio a favor de más elementos nuevos o más antiguos.

En un nivel alto, el uso de la nueva receta de personalización del USUARIO implica los siguientes pasos:

1. Configurar cliente personalizado
2. Crear DatasetGroup, definir un modelo, importar conjuntos de datos e ingesta de interacciones real-time
3. Cree una campaña con una nueva configuración. 'campaignConfig'
4. Cree un rastreador de eventos para la ingesta de los eventos enviados por PutEvents.
5. Llame a GetRecommendations, se devuelve un nuevo campo Recomendación ID en la respuesta.
6. Llame a putEvents con RecommendationsId o una lista personalizada de elementos de impresión.
7. Espere a que se actualice la campaña.
8. Actualizar campaña para detener la actualización automática.
9. Limpiar

### Credenciales AWS Cli.

Las credenciales de AWS deben tener permisos programaticos y a los recursos de la arquitectura definida.

In [2]:
accessKeyId = "clave"
secretAccessKey = "clave"
region_name = "region"

In [49]:
import os
import boto3
from botocore.exceptions import ClientError
import time
import numpy as np
import pandas as pd
import json
from datetime import datetime

In [50]:
suffix = str(np.random.uniform())[4:9]
prefix = 'user-personalization-'
print('prefix+suffix:{}{}'.format(prefix, suffix))
s3_bucket_name = (prefix + suffix).lower()
interaction_schema_name = prefix + 'interaction-'  + suffix
item_metadata_schema_name = prefix + 'items-'  + suffix
dataset_group_name = prefix + suffix
interaction_dataset_name = prefix + 'interactions-' + suffix
item_metadata_dataset_name = prefix + 'items-' + suffix
event_tracker_name = prefix + suffix
solution_name = prefix + suffix
event_tracker_name = prefix + suffix
campaign_name = prefix + suffix

prefix+suffix:user-personalization-62542


### 1. Configuracion del cliente

In [51]:
s3_client = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('s3')

####  Initialize personalize clients

In [52]:
personalize = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize')
personalize_runtime = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize-runtime')
personalize_events = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize-events')

### Datasets Liphycos S.A.

Usaremos los conjuntos de datos de Liphycos S.A. Existen dos conjuntos de datos para este caso, el uno son los metadatos de los items o productos y el otro es un conjunto de datos de interacción del usuario.

In [7]:
interactions_file = os.getcwd() + "/interaction.csv"
items_metadata_file = os.getcwd() + "/items.csv"

In [8]:
interactions_df = pd.read_csv(interactions_file)
items_df = pd.read_csv(items_metadata_file)

#### Echemos un vistazo al marco de datos de interacción 

In [9]:
interactions_df.head(2)

,EVENT_TYPE,IMPRESSION,ITEM_ID,TIMESTAMP,USER_ID
0,click,73|70|117|95|96|92|55|45|116|97|56|54|33|94|36...,73,1613535227,35
1,click,51|67|53|110|61|69|25|49|37|108|116|34|74|99|6...,53,1613538733,57


#### Dataset de Interactiones
**ITEM_ID**: Elemento o producto correspondiente al EVENT_TYPE.

**EVENT_TYPE**: Tipo de evento.

**TIMESTAMP**: Marca de tiempo de la Interacción en milisegundos.

**USER_ID**: ID de usuario correspondiente a esta impresión.

**IMPRESSION:** Ahora, opcionalmente, puede pasar datos de impresiones junto con los datos de eventos en el conjunto de datos de interacción. Esto se pasa en un nuevo campo "IMPRESSION", como puede ver arriba, que toma una concatenación de los elementos con los que el usuario interactuó (por ejemplo, los elementos que se mostraron al usuario). Las impresiones también incluyen los elementos en los que se hizo clic. 


In [10]:
items_df.head(2)

,ITEM_ID,compos,creation_timestamp
0,44,clorhexidina|te-verde|semilla-de-arroz|conserv...,1582077086
1,169,"tensoactivo-biodegradable|ph-5,5|mix-humectant...",1582077086


#### Dataset de Items

El conjunto de datos de elementos contiene Item_ids y metadatos asociados.

**ITEM_ID**: ID de artículo

**compos** Metadatos del item, si hay varios componentes para el mismo elemento, (barra vertical "|" para concatenar).

**creation_timestamp** Marca de tiempo cuando se agregó el elemento 

#### Actualizacion del timestamp

Aquí vamos a actualizar el timestamp de nuestro dataset a 8 días a partir de hoy para mostrar el impacto de las interacciones en tiempo real en nuestras recomendaciones. 

In [11]:
current_time = int(time.time())
one_hour_ago = current_time - 8 * 24 * 60 * 60
# Get the time gap between the latest timestamp in the interaction and the current time 
interactions_df = interactions_df.astype({"TIMESTAMP": 'int64'})
latest_time_in_csv = interactions_df["TIMESTAMP"].max()
delta = one_hour_ago - latest_time_in_csv

In [12]:
# shift the latest timestamp in the interactions_df to be the last hour timestamp
interactions_df.TIMESTAMP = interactions_df.TIMESTAMP + delta
interactions_df.to_csv(os.getcwd() + "/interaction.csv", index = False)

# shift the latest timestamp in the items_df to be the last hour timestamp
items_df = items_df.astype({"creation_timestamp": 'int64'})
items_df.creation_timestamp = items_df.creation_timestamp + delta
items_df.to_csv(os.getcwd() + "/items.csv", index = False)

#### Revisamos nuevamente los datasets

In [13]:
interactions_df.head(2)

,EVENT_TYPE,IMPRESSION,ITEM_ID,TIMESTAMP,USER_ID
0,click,73|70|117|95|96|92|55|45|116|97|56|54|33|94|36...,73,1613568250,35
1,click,51|67|53|110|61|69|25|49|37|108|116|34|74|99|6...,53,1613571756,57


In [14]:
items_df.head(2)

,ITEM_ID,compos,creation_timestamp
0,44,clorhexidina|te-verde|semilla-de-arroz|conserv...,1582110109
1,169,"tensoactivo-biodegradable|ph-5,5|mix-humectant...",1582110109



### 2. Ingesta de data a Amazon Personalize
Ahora vamos a crear un grupo de conjuntos de datos, crear un Schema, cargar los datasets, crear un trabajo o datasetImport Job.

#### a. Crear un DatasetGroup

In [15]:
create_dataset_group_response = personalize.create_dataset_group(
    name = dataset_group_name
)
dataset_group_arn = create_dataset_group_response['datasetGroupArn']

In [16]:
print('dataset_group_arn : {}'.format(dataset_group_arn))

dataset_group_arn : arn:aws:personalize:us-east-1:090793147312:dataset-group/user-personalization-41753


In [17]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


#### b. Crear un Schema Dataset para los items y las interacciones

> **_NOTE:_**: `Impression` field has a type String and uses a piped concatination for multiple values.

In [18]:
interaction_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        { 
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "IMPRESSION",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
    ],
    "version": "1.0"
}

In [19]:
interaction_schema_response = personalize.create_schema(
    name = interaction_schema_name,
    schema = json.dumps(interaction_schema)
)
interaction_schema_arn = interaction_schema_response['schemaArn']
print('interaction_schema_arn:\n', interaction_schema_arn)

interaction_schema_arn:
 arn:aws:personalize:us-east-1:090793147312:schema/user-personalization-interaction-41753


In [20]:
item_metadata_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
    {
        "name": "ITEM_ID",
        "type": "string"
    },
    {
        "name": "COMPOS",
        "type": "string",
        "categorical": True
    },
    {
        "name": "CREATION_TIMESTAMP",
        "type": "long"
    }
    ],
    "version": "1.0"
}

item_metadata_schema_response = personalize.create_schema(
    name = item_metadata_schema_name,
    schema = json.dumps(item_metadata_schema)
)

item_metadata_schema_arn = item_metadata_schema_response['schemaArn']
print('item_metadata_schema_arn:\n', item_metadata_schema_arn)

item_metadata_schema_arn:
 arn:aws:personalize:us-east-1:090793147312:schema/user-personalization-items-41753


#### c. Crear los Datasets

In [21]:
interactions_dataset_response = personalize.create_dataset(
    datasetType = 'INTERACTIONS',
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn,
    name = interaction_dataset_name
)
interaction_dataset_arn = interactions_dataset_response['datasetArn']

print('interaction_dataset_arn:\n', interaction_dataset_arn)

items_dataset_response = personalize.create_dataset(
    datasetType = 'ITEMS',
    datasetGroupArn = dataset_group_arn,
    schemaArn = item_metadata_schema_arn,
    name = item_metadata_dataset_name
)
item_metadata_dataset_arn = items_dataset_response['datasetArn']

print('item_metadata_dataset_arn:\n', item_metadata_dataset_arn)

interaction_dataset_arn:
 arn:aws:personalize:us-east-1:090793147312:dataset/user-personalization-41753/INTERACTIONS
item_metadata_dataset_arn:
 arn:aws:personalize:us-east-1:090793147312:dataset/user-personalization-41753/ITEMS


#### d. Cargar los datasets al S3 bucket, configuracion S3 Bucket policy, IAM Role, etc.,

Necesitamos cargar estos datasets o puede proporcionar el nombre del bucket donde ya tiene los conjuntos de datos 

In [22]:
s3_bucket_name
print(region_name)

us-east-1


In [23]:
s3_client.create_bucket(Bucket=s3_bucket_name,
                          CreateBucketConfiguration={
                              'LocationConstraint': 'eu-west-1'})

{'ResponseMetadata': {'RequestId': '089D0CCFDC824DA3',
  'HostId': 'b12YRq4XNwxKr8DDy5pddfOa2N0vAAhyqsd6SNYdBzF10TdfzWVCEPk3IBn8k2bvBEjSv4H4jJ8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'b12YRq4XNwxKr8DDy5pddfOa2N0vAAhyqsd6SNYdBzF10TdfzWVCEPk3IBn8k2bvBEjSv4H4jJ8=',
   'x-amz-request-id': '089D0CCFDC824DA3',
   'date': 'Thu, 25 Feb 2021 17:42:55 GMT',
   'location': 'http://user-personalization-41753.s3.amazonaws.com/',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': 'http://user-personalization-41753.s3.amazonaws.com/'}

In [24]:
interactions_file = os.getcwd() + "/interaction.csv"
items_metadata_file = os.getcwd() + "/items.csv"

In [25]:
s3_client.upload_file(Filename=interactions_file, Bucket=s3_bucket_name,
    Key="interaction.csv")
s3_client.upload_file(Filename=items_metadata_file, Bucket=s3_bucket_name,
    Key="items.csv")

#### e. Agregar politicas de seguridad al bucket S3 

In [26]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(s3_bucket_name),
                "arn:aws:s3:::{}/*".format(s3_bucket_name)
            ]
        }
    ]
}

s3_client.put_bucket_policy(Bucket=s3_bucket_name, Policy=json.dumps(policy));

#### f. Configurar el rol IAM para poder acceder a los datasets

In [27]:
iam = boto3.client(service_name='iam', 
                         aws_access_key_id = accessKeyId, 
                         aws_secret_access_key = secretAccessKey)   


role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise

In [28]:
print('role_arn:', role_arn)

role_arn: arn:aws:iam::090793147312:role/PersonalizeS3Role-41753


#### g. Crear DatasetImportJobs para cargar la data

In [29]:
time.sleep(20) # wait for RoleARN completion
interactions_dij_response = personalize.create_dataset_import_job(
    jobName =  prefix + 'interactions-dij-' + suffix,
    datasetArn = interaction_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(s3_bucket_name, 'interaction.csv')
    },
    roleArn = role_arn
)

interactions_dij_arn = interactions_dij_response['datasetImportJobArn']
print('interactions_dij_arn: ', interactions_dij_arn)
#print(json.dumps(interactions_dij_arn, indent=2))

items_dij_response = personalize.create_dataset_import_job(
    jobName =  prefix + 'items-dij-' + suffix,
    datasetArn = item_metadata_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(s3_bucket_name, 'items.csv')
    },
    roleArn = role_arn
)

items_dij_arn = items_dij_response['datasetImportJobArn']
print('items_dij_arn:', items_dij_arn)

interactions_dij_arn:  arn:aws:personalize:us-east-1:090793147312:dataset-import-job/user-personalization-interactions-dij-41753
items_dij_arn: arn:aws:personalize:us-east-1:090793147312:dataset-import-job/user-personalization-items-dij-41753


In [30]:
dataset_job_arns = [interactions_dij_arn, items_dij_arn]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time and len(dataset_job_arns) != 0:
    time.sleep(60)    
    for dij_arn in dataset_job_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = dij_arn
        )
        dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
        status = None
        if "latestDatasetImportJobRun" not in dataset_import_job:
            status = dataset_import_job["status"]
            print("{} : {}".format(dij_arn, status))
        else:
            status = dataset_import_job["latestDatasetImportJobRun"]["status"]
            print("DIJ_ARN: {}, LatestDatasetImportJobRun: {}".format(dij_arn, status))
    
        if status == "ACTIVE" or status == "CREATE FAILED":
            dataset_job_arns.remove(dij_arn)


arn:aws:personalize:us-east-1:090793147312:dataset-import-job/user-personalization-interactions-dij-41753 : CREATE IN_PROGRESS
arn:aws:personalize:us-east-1:090793147312:dataset-import-job/user-personalization-items-dij-41753 : CREATE IN_PROGRESS
arn:aws:personalize:us-east-1:090793147312:dataset-import-job/user-personalization-interactions-dij-41753 : CREATE IN_PROGRESS
arn:aws:personalize:us-east-1:090793147312:dataset-import-job/user-personalization-items-dij-41753 : CREATE IN_PROGRESS
arn:aws:personalize:us-east-1:090793147312:dataset-import-job/user-personalization-interactions-dij-41753 : CREATE IN_PROGRESS
arn:aws:personalize:us-east-1:090793147312:dataset-import-job/user-personalization-items-dij-41753 : CREATE IN_PROGRESS
arn:aws:personalize:us-east-1:090793147312:dataset-import-job/user-personalization-interactions-dij-41753 : CREATE IN_PROGRESS
arn:aws:personalize:us-east-1:090793147312:dataset-import-job/user-personalization-items-dij-41753 : CREATE IN_PROGRESS
arn:aws:pers

print(os.getcwd())

### 3. Crear la Solucion y la Version de la Solution

Crearemos una solución con 'aws-user-personalization'. Esta receta equilibra las recomendaciones para artículos nuevos y antiguos entregados a los usuarios 

In [54]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"
max_time = time.time() + 3*60*60 # 3 hours
create_solution_response = None
while time.time() < max_time:

    try:
        create_solution_response = personalize.create_solution(name=solution_name, 
                                    recipeArn= recipe_arn, 
                                    datasetGroupArn = dataset_group_arn)

        solution_arn = create_solution_response['solutionArn']
        print('solution_arn: ', solution_arn)
        break;
    except personalize.exceptions.ClientError as e:
        if 'EVENT_INTERACTIONS' not in str(e):
            print(json.dumps(create_solution_response, indent=2))
            print(e)
            break

null
An error occurred (InvalidSignatureException) when calling the CreateSolution operation: Signature expired: 20210225T205002Z is now earlier than 20210225T205926Z (20210225T210426Z - 5 min.)


In [32]:
create_solution_version_response = personalize.create_solution_version(solutionArn = solution_arn)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print('solution_version_arn:', solution_version_arn)

solution_version_arn: arn:aws:personalize:us-east-1:090793147312:solution/user-personalization-41753/0f2db750


In [35]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_P

In [53]:
solutionMetrics = personalize.get_solution_metrics(
    solutionVersionArn=solution_version_arn
)
print('solutionMetrics',solutionMetrics["metrics"])

ClientError: An error occurred (InvalidSignatureException) when calling the GetSolutionMetrics operation: Signature expired: 20210225T204946Z is now earlier than 20210225T205910Z (20210225T210410Z - 5 min.)

### 4. Crear la campaña
Al crear la campaña, podemos configurar itemExplorationConfig para configurar el peso de exploración de items fríos (cold-items) y también el límite de tiempo de exploración (cut-off). Por ahora, podemos establecer un valor de exploración más alto en 0.9 y exploraciónItemAgeCutOff en 7, por lo que creemos que todo el tiempo de creación de elementos de menos de 7 días se consideraría un elemento frío; haríamos más exploración en esos elementos nuevos. 

#### Crear la campaña

In [37]:
create_campaign_response = personalize.create_campaign(
    name = prefix + suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.9",
            "explorationItemAgeCutOff": "7"
        }
    }
)

campaign_arn = create_campaign_response['campaignArn']
print('campaign_arn:', campaign_arn)

campaign_arn: arn:aws:personalize:us-east-1:090793147312:campaign/user-personalization-41753


In [39]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


In [40]:
describe_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)
campaign_summary = describe_campaign_response["campaign"]
campaign_summary

{'name': 'user-personalization-41753',
 'campaignArn': 'arn:aws:personalize:us-east-1:090793147312:campaign/user-personalization-41753',
 'solutionVersionArn': 'arn:aws:personalize:us-east-1:090793147312:solution/user-personalization-41753/0f2db750',
 'minProvisionedTPS': 1,
 'campaignConfig': {'itemExplorationConfig': {'explorationItemAgeCutOff': '7',
   'explorationWeight': '0.9'}},
 'status': 'ACTIVE',
 'creationDateTime': datetime.datetime(2021, 2, 25, 14, 18, 26, 507000, tzinfo=tzlocal()),
 'lastUpdatedDateTime': datetime.datetime(2021, 2, 25, 14, 26, 20, 717000, tzinfo=tzlocal())}

### 5. Llamar al API GetRecommendations
Usaremos los USER_IDs en el conjunto de datos de entrada para realizar llamadas getRecommendation. 
> **NOTA:**: En la respuesta, tiene un nuevo campo `RecommendationId` que corresponde a la lista de items devueltos al Personalizar GetRecommendations. Puede pasar este ID de recomendación para indicar las impresiones.
También puede pasar Impression como una concatenación de elementos de cadena canalizada, si pasa tanto el ID de recomendación como el ImpressionList, ImpressionList tendría prioridad y se usaría en el sistema. 

In [41]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '30', numResults = 10)
print(rec_response['recommendationId'])

RID-787d6ee9-6060-4641-b72b-60ffda085101


In [42]:
rec_response['itemList']

[{'itemId': '30', 'score': 0.0341857},
 {'itemId': '88', 'score': 0.0286532},
 {'itemId': '55', 'score': 0.0272921},
 {'itemId': '27', 'score': 0.0238826},
 {'itemId': '69', 'score': 0.0237205},
 {'itemId': '31', 'score': 0.0229235},
 {'itemId': '28', 'score': 0.0214466},
 {'itemId': '93', 'score': 0.0173127},
 {'itemId': '80', 'score': 0.0167786},
 {'itemId': '90', 'score': 0.0166165}]

### 6. Crear Event Tracker

Crea un event tracker que se usa cuando envía datos de eventos al grupo de conjunto de datos especificado usando la API PutEvents. 

In [43]:
even_tracker_response = personalize.create_event_tracker( 
    name=event_tracker_name,
    datasetGroupArn=dataset_group_arn
)
event_tracker_arn  = even_tracker_response['eventTrackerArn']
event_tracking_id = even_tracker_response['trackingId']
#print(json.dumps(even_tracker_response,indent=2))
print('eventTrackerArn:{},\n eventTrackingId:{}'.format(event_tracker_arn, event_tracking_id))

eventTrackerArn:arn:aws:personalize:us-east-1:090793147312:event-tracker/019f260a,
 eventTrackingId:fa4e077d-72a1-4b3a-b7c8-ebdf67560921


### 7. Enviar datos de Impression a Amazon Personalize via PutEvents.
Amazon Personalize puede modelar dos tipos de impresiones:
1. Impresiones implícitas e impresiones explícitas. Las impresiones implícitas son impresiones que se producen durante la sesión de un usuario y Amazon Personalize las registra automáticamente cada vez que se muestra un artículo al usuario. Puede integrarlos en su flujo de trabajo de recomendaciones incluyendo el ID de recomendación (devuelto por las operaciones y) como entrada para futuras solicitudes de PutEvents.


2. Las impresiones explícitas son impresiones que ingresa manualmente al realizar una solicitud PutEvents. Utilizaría Impresiones explícitas cuando, por ejemplo, no muestre algunos de los artículos devueltos por GetRecommendations debido a la falta de disponibilidad, etc.

> **NOTA:** Si ha definido "impresión" en su esquema de interacción como se indicó anteriormente, debe enviar la lista de impresiones (ya sea los elementos devueltos por GetRecommendations o los suyos propios).
** Cuando tanto el ID de recomendación como las Impresiones son, Amazon Personalize utilizará las impresiones explícitas de forma predeterminada. ** 

#### Pongamos el item recomendado anteriormente como impresiones. 

In [44]:
personalize_events.put_events(
     trackingId = event_tracking_id,
     userId= '30',
     sessionId = '1',
     eventList = [{
     'sentAt': datetime.now().timestamp(),
     'eventType' : 'click',
     'itemId' : rec_response['itemList'][0]['itemId'],        
     'recommendationId': rec_response['recommendationId'],
     'impression': [item['itemId'] for item in rec_response['itemList']],
     }]
    )

{'ResponseMetadata': {'RequestId': '45856655-b4c5-4f67-9375-c6a090aaaf85',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Thu, 25 Feb 2021 19:35:40 GMT',
   'x-amzn-requestid': '45856655-b4c5-4f67-9375-c6a090aaaf85',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

#### También podemos poner algunos elementos nuevos. 

Pongamos el nuevo itemId '3xx' en la personalización. 

In [45]:
personalize_events.put_events(
     trackingId = event_tracking_id,
     userId= '30',
     sessionId = '1',
     eventList = [{
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '340',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '341',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '342',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '343',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '344',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '345',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '346',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '347',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '348',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '349',
     }]
    )

{'ResponseMetadata': {'RequestId': '77eb5814-5ce3-42ac-9058-d396d6185fd4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Thu, 25 Feb 2021 19:36:21 GMT',
   'x-amzn-requestid': '77eb5814-5ce3-42ac-9058-d396d6185fd4',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

### 8. Crear una nueva SolutionVersion con updateMode

Después de poner los eventos, espere unos 15 minutos para que Personalize ingiera los nuevos datos, luego cree una nueva solución Versión con modo de actualización 

In [47]:
create_solution_version_response = personalize.create_solution_version(solutionArn = solution_arn, trainingMode = "UPDATE")

solution_version_after_update = create_solution_version_response['solutionVersionArn']
print('solution_version_after_update:', solution_version_arn)

solution_version_after_update: arn:aws:personalize:us-east-1:090793147312:solution/user-personalization-41753/0f2db750


In [54]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_after_update
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

### 9. Actualizando Campaña

Actualice la campaña con la última versión de la solución arn de la actualización. 

In [56]:
campaign_arn_response = personalize.update_campaign(campaignArn=campaign_arn, solutionVersionArn=solution_version_after_update)
print('campaign_arn_response: ', campaign_arn_response)

campaign_arn_response:  {'campaignArn': 'arn:aws:personalize:us-east-1:090793147312:campaign/user-personalization-33727', 'ResponseMetadata': {'RequestId': '142b9841-f564-432a-8cd4-5c4289cc146e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Thu, 25 Feb 2021 02:11:54 GMT', 'x-amzn-requestid': '142b9841-f564-432a-8cd4-5c4289cc146e', 'content-length': '96', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [57]:
# Wait for campaign update to reflect the new solution-version
solutionVersionArn = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    solutionVersionArn = describe_campaign_response["campaign"]["solutionVersionArn"]
    print("Campaign solution version: {}".format(solutionVersionArn))
    
    if solutionVersionArn == solution_version_after_update:
        break
        
    time.sleep(60)

# wait 1 minutes
time.sleep(60)

Campaign solution version: arn:aws:personalize:us-east-1:090793147312:solution/user-personalization-33727/871da2cb
Campaign solution version: arn:aws:personalize:us-east-1:090793147312:solution/user-personalization-33727/871da2cb
Campaign solution version: arn:aws:personalize:us-east-1:090793147312:solution/user-personalization-33727/871da2cb
Campaign solution version: arn:aws:personalize:us-east-1:090793147312:solution/user-personalization-33727/871da2cb
Campaign solution version: arn:aws:personalize:us-east-1:090793147312:solution/user-personalization-33727/871da2cb
Campaign solution version: arn:aws:personalize:us-east-1:090793147312:solution/user-personalization-33727/871da2cb
Campaign solution version: arn:aws:personalize:us-east-1:090793147312:solution/user-personalization-33727/871da2cb
Campaign solution version: arn:aws:personalize:us-east-1:090793147312:solution/user-personalization-33727/871da2cb
Campaign solution version: arn:aws:personalize:us-east-1:090793147312:solution/u

In [58]:
desc_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)['campaign']["solutionVersionArn"]
desc_campaign_response

'arn:aws:personalize:us-east-1:090793147312:solution/user-personalization-33727/ea6bdef6'

### Después de actualizar la versión de la solución, hagamos la recomendación nuevamente. 

Esperamos que se muestren nuevos elementos en la recomendación, ya que establecemos la exploración alta en 0.9 

In [59]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')

In [60]:
rec_response['itemList']

[{'itemId': '240', 'score': 0.2856446},
 {'itemId': '249', 'score': 0.2368072},
 {'itemId': '245', 'score': 0.2039357},
 {'itemId': '246', 'score': 0.1362455},
 {'itemId': '241', 'score': 0.1355392},
 {'itemId': '243', 'score': 0.0002408},
 {'itemId': '247', 'score': 7.43e-05},
 {'itemId': '55', 'score': 7.13e-05},
 {'itemId': '30', 'score': 5.63e-05},
 {'itemId': '27', 'score': 4.22e-05},
 {'itemId': '16', 'score': 3.58e-05},
 {'itemId': '69', 'score': 3.49e-05},
 {'itemId': '40', 'score': 3.45e-05},
 {'itemId': '80', 'score': 3.36e-05},
 {'itemId': '31', 'score': 3.36e-05},
 {'itemId': '88', 'score': 3.23e-05},
 {'itemId': '42', 'score': 3.16e-05},
 {'itemId': '81', 'score': 3.11e-05},
 {'itemId': '13', 'score': 3.1e-05},
 {'itemId': '12', 'score': 3.07e-05},
 {'itemId': '90', 'score': 3.02e-05},
 {'itemId': '242', 'score': 2.83e-05},
 {'itemId': '19', 'score': 2.62e-05},
 {'itemId': '72', 'score': 2.51e-05},
 {'itemId': '93', 'score': 2.33e-05}]

### Actualización de campaña con diferente explorationWeight

We would expect more old items show in the recommendation list since we set a low explorationWeight.

In [61]:
desc_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)['campaign']
desc_campaign_response

{'name': 'user-personalization-33727',
 'campaignArn': 'arn:aws:personalize:us-east-1:090793147312:campaign/user-personalization-33727',
 'solutionVersionArn': 'arn:aws:personalize:us-east-1:090793147312:solution/user-personalization-33727/ea6bdef6',
 'minProvisionedTPS': 1,
 'campaignConfig': {'itemExplorationConfig': {'explorationItemAgeCutOff': '7',
   'explorationWeight': '0.9'}},
 'status': 'ACTIVE',
 'creationDateTime': datetime.datetime(2021, 2, 24, 19, 25, 58, 212000, tzinfo=tzlocal()),
 'lastUpdatedDateTime': datetime.datetime(2021, 2, 24, 21, 19, 49, 633000, tzinfo=tzlocal()),
 'latestCampaignUpdate': {'solutionVersionArn': 'arn:aws:personalize:us-east-1:090793147312:solution/user-personalization-33727/ea6bdef6',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2021, 2, 24, 21, 11, 55, 117000, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2021, 2, 24, 21, 24, 55, 522000, tzinfo=tzlocal())}}

In [62]:
campaign_arn_response = personalize.update_campaign(campaignArn=campaign_arn, campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.1",
            "explorationItemAgeCutOff": "7"
        }
    })


In [63]:
# Wait for campaign update to reflect the new explorationWeight
explorationWeight = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    explorationWeight = describe_campaign_response["campaign"]["campaignConfig"]['itemExplorationConfig']['explorationWeight']
    print("Current Campaign  explorationWeight: {}".format(explorationWeight))
    
    if explorationWeight == "0.1":
        break
        
    time.sleep(60)

# wait 1 minutes
time.sleep(60)

Current Campaign  explorationWeight: 0.9
Current Campaign  explorationWeight: 0.1


### Después de actualizado explorationWeight

Hagamos una recomendación nuevamente, deberíamos ver más elementos antiguos aquí. 

In [64]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')

In [65]:
rec_response

{'ResponseMetadata': {'RequestId': '62fa48e1-db3f-44a7-9f75-e6208286279b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Thu, 25 Feb 2021 02:50:48 GMT',
   'x-amzn-requestid': '62fa48e1-db3f-44a7-9f75-e6208286279b',
   'content-length': '1393',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'itemList': [{'itemId': '55', 'score': 0.0447419},
  {'itemId': '30', 'score': 0.0353188},
  {'itemId': '27', 'score': 0.02648},
  {'itemId': '16', 'score': 0.0224729},
  {'itemId': '69', 'score': 0.0219123},
  {'itemId': '40', 'score': 0.0216392},
  {'itemId': '80', 'score': 0.0210774},
  {'itemId': '31', 'score': 0.0210631},
  {'itemId': '88', 'score': 0.0202834},
  {'itemId': '42', 'score': 0.0198076},
  {'itemId': '81', 'score': 0.0195448},
  {'itemId': '13', 'score': 0.019479},
  {'itemId': '12', 'score': 0.0193007},
  {'itemId': '90', 'score': 0.0189647},
  {'itemId': '19', 'score': 0.0164296},
  {'itemId': '72', 'score': 0.015745},
  {'ite

#### 10. Eliminar Recursos en AWS

After created all resouces, let's cleanup all resources.

In [1]:
personalize.delete_campaign(campaignArn=campaign_arn)
while len(personalize.list_campaigns(solutionArn=solution_arn)['campaigns']):
    time.sleep(5)

personalize.delete_solution(solutionArn=solution_arn)
while len(personalize.list_solutions(datasetGroupArn=dataset_group_arn)['solutions']):
    time.sleep(5)

for dataset in personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']:
    personalize.delete_dataset(datasetArn=dataset['datasetArn'])
while len(personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']):
    time.sleep(5)
    
personalize.delete_event_tracker(eventTrackerArn=event_tracker_arn)
personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)

NameError: name 'personalize' is not defined